In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np

In [ ]:
intervals = [0.01, 0.05, 0.1, 0.5, 1]
burstsize = 4 

fig, ax = plt.subplots(figsize=(16, 10))

#df_delay = pd.read_csv("delays.csv", names=["t", "delay"])
#df_delay["rtt"] = 2 * df_delay["delay"]
#df_delay.rolling(10).median().plot(x='t', y='rtt', color='lightgray', ax=ax)

for interval in intervals:
#for burstsize in [1, 2, 3, 4,5, 6, 7, 10]:
    params = (interval, burstsize)
    df = pd.read_csv("rtt[%s,%s].csv" % params, names=["t", "burst", "rtt"])
    df = df.groupby("burst").median()
    df_delay = pd.read_csv("delays[%s,%s].csv" % params, names=["t", "delay"])
    
    baseline = 2 * np.interp(df['t'], df_delay['t'], df_delay['delay'])
    df["error"] = df['rtt'] - baseline
    df.plot(x='t', y='error', ax=ax, label=str(interval))

fig.tight_layout()
ax.legend()
plt.show()

In [ ]:
burstsizes = [1, 2, 5, 10, 20]
burstsizes = [1, 4, 10]
interval = 0.1

def plot_rtt(df, ax, label, window=1):
    df = df.groupby("burst").agg(["median", 'min', 'max'])
    t = df[('t', 'max')]
    rtt = df[('rtt', 'median')]
    rtt_min = df[('rtt', 'min')]
    rtt_max = df[('rtt', 'max')]

    lines = ax.plot(t, rtt, label=str(label))
    color = lines[0].get_color()
    ax.fill_between(t, rtt_min, rtt_max, color=color, alpha=0.2)


fig, ax = plt.subplots(figsize=(16, 10))

# Plot actual delay measurements in the background
df_delay = pd.read_csv("delays.csv", names=["t", "delay"])
# Filter away startup phase
df_delay = df_delay[df_delay['t'] > 2.5]
df_delay["rtt"] = 2 * df_delay["delay"]
df_delay.rolling(10).median().plot(x='t', y='rtt', color='lightgray', ax=ax)



for burstsize in burstsizes:
    df = pd.read_csv("rtt[%s,%s].csv" % (interval, burstsize),
                     names=["t", "burst", "rtt"])
            
    # Filter away startup phase
    df = df[df['t'] > 2.5]
    
    plot_rtt(df, ax, burstsize)


fig.tight_layout()
ax.legend()
plt.show()

In [ ]:
def mse(burstsize, interval=0.1, metric='median', df_delay=df_delay):
    df_delay = pd.read_csv("delays[%s,%s].csv" % (interval, burstsize),
                           names=["t", "delay"])
    df_delay["rtt"] = 2 * df_delay["delay"]

    df = pd.read_csv("rtt[%s,%s].csv" % (interval, burstsize),
                     names=["t", "burst", "rtt"])
    df = df[df['t'] > 2.5]

    df = df.groupby("burst").agg([metric, 'min', 'max'])
    ground_truth = np.interp(df[('t', 'max')], df_delay['t'], df_delay['rtt'])
    measures = df[('rtt', metric)]
    return np.sqrt(np.sum(np.power(measures - ground_truth, 2)))

burstsizes = [1, 2, 3, 4, 5, 6, 7, 10]
errors = [mse(bs, metric='median') for bs in burstsizes]
plt.plot(burstsizes, errors)
plt.show()